<a href="https://colab.research.google.com/github/AlineSchaub/DataScience21/blob/7_Block/Kopie_von_Exercise_2_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment - Image Classification with CNNs
![CIFAR10](https://miro.medium.com/max/964/1*syyml8q8s1Yt-iEea5m1Ag.png)

[CIFAR 10](https://www.cs.toronto.edu/~kriz/cifar.html) is a small toy data set for image classifaction. 

* It has 60k images from ten diffenrent classes (like car, airplane, horse,..)
* The RGB images are of size 32x32 pixels  

In [1]:
#import needed libs
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

#some vars to controll the training -> use them in your code
batch_size = 32
num_classes = 10 #fixed!
epochs = 10

# Get the data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 2s 0us/step
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


## Task:
* visualize some sample images from different classes
* build and train the ***AlexNet*** CNN architecture from the lecture, use the following Keras layers:
    * overview: https://keras.io/api/layers/
    * 2d convolution: https://keras.io/api/layers/convolution_layers/convolution2d/
    * Fully Connected (=Matrix Multiplication MLP): https://keras.io/api/layers/core_layers/dense/
    * Pooling: https://keras.io/api/layers/pooling_layers/max_pooling2d/   
* use 'relu' functions for the non-linear activation -> https://keras.io/api/layers/activation_layers/relu/
* optimize the learning hyper-paramters to get the best  test accuracy

In [2]:
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import CenterCrop
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras import activations

#RGB (3) und Bildgröße 32 auf 32
inputs = keras.Input(shape=(32, 32, 3))


#Bilder zuschneiden
x_1 = CenterCrop(height=30, width=30)(inputs)


# Rescale images to [0, 1]
x_2 = Rescaling(scale=1.0 / 255)(x_1)

# 2d Convolution layer
x_3 = layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu")(x_2)

#fully connected layer
x_4 = layers.Dense(32, activation='relu')(x_3)

#Pooling layer
x_5 = layers.MaxPooling2D(pool_size=(3, 3))(x_4)


x_6 = layers.GlobalAveragePooling2D()(x_5)



In [3]:

outputs = layers.Dense(num_classes, activation=activations.relu)(x_6)

In [4]:
model = keras.Model(inputs=inputs, outputs=outputs)

In [5]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
center_crop (CenterCrop)     (None, 30, 30, 3)         0         
_________________________________________________________________
rescaling (Rescaling)        (None, 30, 30, 3)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 32)        896       
_________________________________________________________________
dense (Dense)                (None, 28, 28, 32)        1056      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 9, 9, 32)          0         
_________________________________________________________________
global_average_pooling2d (Gl (None, 32)                0     

In [6]:
#optimize + loss
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
              loss=keras.losses.CategoricalCrossentropy())

In [7]:
#batch size und epochs von oben

history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)

Epoch 1/10
1563/1563 [==============================] - 39s 24ms/step - loss: 3.6775
Epoch 2/10
1563/1563 [==============================] - 36s 23ms/step - loss: 3.4235
Epoch 3/10
1563/1563 [==============================] - 37s 24ms/step - loss: 3.4352
Epoch 4/10
1563/1563 [==============================] - 35s 23ms/step - loss: 3.4025
Epoch 5/10
1563/1563 [==============================] - 36s 23ms/step - loss: 3.4459
Epoch 6/10
1563/1563 [==============================] - 37s 24ms/step - loss: nan
Epoch 7/10
1563/1563 [==============================] - 36s 23ms/step - loss: nan
Epoch 8/10
1563/1563 [==============================] - 35s 22ms/step - loss: nan
Epoch 9/10
1563/1563 [==============================] - 35s 23ms/step - loss: nan
Epoch 10/10
1563/1563 [==============================] - 35s 22ms/step - loss: nan


In [8]:
print(history.history)

{'loss': [3.5112996101379395, 3.413945198059082, 3.4052655696868896, 3.409304618835449, 3.443108081817627, nan, nan, nan, nan, nan]}


In [18]:
#Optimize+loss+accuracy
model2 = keras.Model(inputs=inputs, outputs=outputs)
model2.compile(optimizer=keras.optimizers.Adam(), loss=keras.losses.CategoricalCrossentropy(), metrics='accuracy')


In [19]:
import tensorflow as tf

val_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

In [20]:
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)


In [21]:
val_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)
history = model2.fit(dataset, epochs=epochs, batch_size=batch_size, validation_data=val_dataset)

Epoch 1/10
1563/1563 [==============================] - 37s 23ms/step - loss: nan - accuracy: 0.0996 - val_loss: nan - val_accuracy: 0.1000
Epoch 2/10
1563/1563 [==============================] - 37s 24ms/step - loss: nan - accuracy: 0.0996 - val_loss: nan - val_accuracy: 0.1000
Epoch 3/10
1563/1563 [==============================] - 37s 24ms/step - loss: nan - accuracy: 0.0996 - val_loss: nan - val_accuracy: 0.1000
Epoch 4/10
1563/1563 [==============================] - 38s 24ms/step - loss: nan - accuracy: 0.0996 - val_loss: nan - val_accuracy: 0.1000
Epoch 5/10
1563/1563 [==============================] - 37s 23ms/step - loss: nan - accuracy: 0.0996 - val_loss: nan - val_accuracy: 0.1000
Epoch 6/10
1563/1563 [==============================] - 39s 25ms/step - loss: nan - accuracy: 0.0996 - val_loss: nan - val_accuracy: 0.1000
Epoch 7/10
1563/1563 [==============================] - 39s 25ms/step - loss: nan - accuracy: 0.0996 - val_loss: nan - val_accuracy: 0.1000
Epoch 8/10
1563/1563

In [22]:
loss, acc = model.evaluate(val_dataset)
print("loss: %.2f" % loss)
print("acc: %.2f" % acc)

313/313 [==============================] - 3s 11ms/step - loss: nan - accuracy: 0.1000
loss: nan
acc: 0.10


In [23]:
predictions = model.predict(val_dataset)
print(predictions)

[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]


In [24]:
#Optimize+loss+accuracy
model3 = keras.Model(inputs=inputs, outputs=outputs)
model3.compile(optimizer=keras.optimizers.Adam(), loss='mse', run_eagerly=True, metrics='accuracy')
history = model3.fit(dataset, epochs=epochs, batch_size=batch_size, validation_data=val_dataset)



Epoch 1/10
1563/1563 [==============================] - 88s 56ms/step - loss: nan - accuracy: 0.0996 - val_loss: nan - val_accuracy: 0.1000
Epoch 2/10
1563/1563 [==============================] - 88s 56ms/step - loss: nan - accuracy: 0.0996 - val_loss: nan - val_accuracy: 0.1000
Epoch 3/10
1563/1563 [==============================] - 78s 50ms/step - loss: nan - accuracy: 0.0996 - val_loss: nan - val_accuracy: 0.1000
Epoch 4/10
1563/1563 [==============================] - 82s 52ms/step - loss: nan - accuracy: 0.0996 - val_loss: nan - val_accuracy: 0.1000
Epoch 5/10
1563/1563 [==============================] - 80s 51ms/step - loss: nan - accuracy: 0.0996 - val_loss: nan - val_accuracy: 0.1000
Epoch 6/10
1563/1563 [==============================] - 81s 52ms/step - loss: nan - accuracy: 0.0996 - val_loss: nan - val_accuracy: 0.1000
Epoch 7/10
1563/1563 [==============================] - 83s 53ms/step - loss: nan - accuracy: 0.0996 - val_loss: nan - val_accuracy: 0.1000
Epoch 8/10
1563/1563

Hyperparameter Tuning

In [69]:
def build_model(hp):
    inputs = keras.Input(shape=(32, 32, 3)),
    x = layers.Dense(
        32, activation='relu')(inputs)
    outputs = layers.Dense(num_classes, activation=activations.relu)(x)
    model = keras.Model(inputs, outputs)
    model.compile(
        optimizer=keras.optimizers.Adam(), loss=keras.losses.CategoricalCrossentropy(), metrics='accuracy')
    return model

   


In [70]:
!pip install -q -U keras-tuner
from kerastuner.tuners import Hyperband


In [84]:
tuner = Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=4)


INFO:tensorflow:Reloading Oracle from existing project ./untitled_project/oracle.json
Invalid model 0/5


Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/kerastuner/engine/hypermodel.py", line 104, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-69-26d710e91ab6>", line 4, in build_model
    32, activation='relu')(inputs)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer.py", line 952, in __call__
    input_list)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer.py", line 1091, in _functional_construction_call
    inputs, input_masks, args, kwargs)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer.py", line 822, in _keras_tensor_symbolic_call
    return self._infer_output_signature(inputs, args, kwargs, input_masks)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer.py", line 862, in _infer_output_signature
    self._maybe_build(inputs)
  File "/usr/local/lib/python3.7/dist-packages/

Invalid model 1/5
Invalid model 2/5
Invalid model 3/5


Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/kerastuner/engine/hypermodel.py", line 104, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-69-26d710e91ab6>", line 4, in build_model
    32, activation='relu')(inputs)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer.py", line 952, in __call__
    input_list)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer.py", line 1091, in _functional_construction_call
    inputs, input_masks, args, kwargs)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer.py", line 822, in _keras_tensor_symbolic_call
    return self._infer_output_signature(inputs, args, kwargs, input_masks)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer.py", line 862, in _infer_output_signature
    self._maybe_build(inputs)
  File "/usr/local/lib/python3.7/dist-packages/

Invalid model 4/5
Invalid model 5/5


Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/kerastuner/engine/hypermodel.py", line 104, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-69-26d710e91ab6>", line 4, in build_model
    32, activation='relu')(inputs)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer.py", line 952, in __call__
    input_list)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer.py", line 1091, in _functional_construction_call
    inputs, input_masks, args, kwargs)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer.py", line 822, in _keras_tensor_symbolic_call
    return self._infer_output_signature(inputs, args, kwargs, input_masks)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer.py", line 862, in _infer_output_signature
    self._maybe_build(inputs)
  File "/usr/local/lib/python3.7/dist-packages/

RuntimeError: ignored

In [51]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [80]:
best_model = tuner.get_best_models(num_models=4)
print(best_model)

[]


In [79]:
best_hyperparameters = tuner.get_best_hyperparameters
print(best_hyperparameters)

<bound method BaseTuner.get_best_hyperparameters of <kerastuner.tuners.hyperband.Hyperband object at 0x7f8e09c631d0>>


In [37]:
tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name='val_loss', direction='min')
